# 11. Additive Postprocessing Experiment

This notebook runs additive postprocessing on synthetic reporting tables without changing existing notebooks or baseline outputs.

Outputs are written to `data/experiments_additive/postprocess_*`.

In [1]:
from pathlib import Path
import sys
import pandas as pd
from IPython.display import display, Markdown

ROOT = Path.cwd().resolve().parent if Path.cwd().name == 'notebooks' else Path.cwd().resolve()
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

from src.experiments.postprocess_reporting import PostprocessConfig, postprocess_reporting_dir
from src.pipeline.run_benchmark import run_all
from src.eval.compare import evaluate_all, results_to_dataframe

REAL_REPORTING_DIR = ROOT / 'data' / 'reporting'
REAL_RESULTS_DIR = ROOT / 'data' / 'results' / 'real'
QUERIES_DIR = ROOT / 'docs' / 'queries'
OUT_ROOT = ROOT / 'data' / 'experiments_additive'

display(Markdown(f"Project root: `{ROOT}`"))

Project root: `/Users/enscribe/Repositories/School/dsc180-q2`

In [2]:
runs = {
    'pertable_refcats': {
        'synth_reporting_dir': ROOT / 'data' / 'reporting' / 'synth_pertable',
        'cfg': PostprocessConfig(use_reference_categories=True, fuzzy_cutoff=0.80),
    },
    'mst_refcats': {
        'synth_reporting_dir': ROOT / 'data' / 'reporting' / 'synth_mst',
        'cfg': PostprocessConfig(use_reference_categories=True, fuzzy_cutoff=0.80),
    },
    'pe_refcats': {
        'synth_reporting_dir': ROOT / 'data' / 'reporting' / 'pe',
        'cfg': PostprocessConfig(use_reference_categories=True, fuzzy_cutoff=0.80),
    },
}

summary_rows = []
for name, spec in runs.items():
    out_dir = OUT_ROOT / f'postprocess_{name}'
    post_dir = out_dir / 'reporting'
    query_dir = out_dir / 'query_results'
    out_dir.mkdir(parents=True, exist_ok=True)

    written = postprocess_reporting_dir(
        real_reporting_dir=REAL_REPORTING_DIR,
        synth_reporting_dir=spec['synth_reporting_dir'],
        output_dir=post_dir,
        cfg=spec['cfg'],
    )

    run_all(
        queries_dir=QUERIES_DIR,
        reporting_dir=post_dir,
        output_dir=query_dir,
        skip_infeasible=True,
        verbose=False,
    )

    eval_results = evaluate_all(REAL_RESULTS_DIR, query_dir)
    eval_df = results_to_dataframe(eval_results)
    eval_df.to_csv(out_dir / 'evaluation.csv', index=False)

    ev = eval_df[eval_df['n_metrics'] > 0]
    passed = int(ev['passed'].fillna(False).sum()) if len(ev) else 0
    avg_score = float(ev['score'].mean()) if len(ev) else 0.0
    summary_rows.append({
        'run': name,
        'tables_written': len(written),
        'queries_evaluated': len(ev),
        'queries_passed': passed,
        'pass_rate': passed / len(ev) if len(ev) else 0.0,
        'avg_score': avg_score,
        'evaluation_csv': str(out_dir / 'evaluation.csv'),
    })

summary = pd.DataFrame(summary_rows).sort_values(['queries_passed', 'avg_score'], ascending=False)
display(Markdown('## Additive run summary'))
display(summary)

  FAIL battery_on_duration_cpu_family_gen: IO Error: No files found that match the pattern "/Users/enscribe/Repositories/School/dsc180-q2/data/experiments_additive/postprocess_pe_refcats/reporting/system_cpu_metadata.parquet"

LINE 1: ..., avg(duration_mins) as avg_duration_mins_on_battery from read_parquet('/Users/enscribe/Repositories/School/dsc180...
                                                                     ^
  FAIL display_devices_connection_type_resolution_durations_ac_dc: IO Error: No files found that match the pattern "/Users/enscribe/Repositories/School/dsc180-q2/data/experiments_additive/postprocess_pe_refcats/reporting/system_display_devices.parquet"

LINE 1: ...(avg(duration_dc),2) as average_duration_on_dc_in_seconds from read_parquet('/Users/enscribe/Repositories/School/dsc180...
                                                                          ^
  FAIL display_devices_vendors_percentage: IO Error: No files found that match the pattern "/Users/enscribe/R

## Additive run summary

,run,tables_written,queries_evaluated,queries_passed,pass_rate,avg_score,evaluation_csv
1,mst_refcats,18,21,6,0.285714,0.328326,/Users/enscribe/Repositories/School/dsc180-q2/...
0,pertable_refcats,19,21,6,0.285714,0.303107,/Users/enscribe/Repositories/School/dsc180-q2/...
2,pe_refcats,12,8,2,0.250000,0.149866,/Users/enscribe/Repositories/School/dsc180-q2/...


In [3]:
baseline_files = {
    'pertable_baseline': ROOT / 'data' / 'results' / 'evaluation_pertable.csv',
    'mst_baseline': ROOT / 'data' / 'results' / 'evaluation_mst.csv',
    'pe_baseline': ROOT / 'data' / 'results' / 'evaluation_pe.csv',
    'widetable_baseline': ROOT / 'data' / 'results' / 'evaluation_widetable.csv',
}

rows = []
for label, path in baseline_files.items():
    if not path.exists():
        continue
    df = pd.read_csv(path)
    ev = df[df['n_metrics'] > 0]
    passed = int(ev['passed'].fillna(False).sum()) if len(ev) else 0
    rows.append({
        'run': label,
        'queries_evaluated': len(ev),
        'queries_passed': passed,
        'pass_rate': passed / len(ev) if len(ev) else 0.0,
        'avg_score': float(ev['score'].mean()) if len(ev) else 0.0,
        'evaluation_csv': str(path),
    })

baseline_summary = pd.DataFrame(rows)
display(Markdown('## Baseline summary'))
display(baseline_summary.sort_values(['queries_passed', 'avg_score'], ascending=False))

## Baseline summary

,run,queries_evaluated,queries_passed,pass_rate,avg_score,evaluation_csv
1,mst_baseline,21,6,0.285714,0.328326,/Users/enscribe/Repositories/School/dsc180-q2/...
0,pertable_baseline,21,6,0.285714,0.303107,/Users/enscribe/Repositories/School/dsc180-q2/...
2,pe_baseline,8,2,0.250000,0.149866,/Users/enscribe/Repositories/School/dsc180-q2/...
3,widetable_baseline,8,1,0.125000,0.258065,/Users/enscribe/Repositories/School/dsc180-q2/...


In [4]:
delta_rows = []
mapping = {
    'pertable_refcats': ROOT / 'data' / 'results' / 'evaluation_pertable.csv',
    'mst_refcats': ROOT / 'data' / 'results' / 'evaluation_mst.csv',
    'pe_refcats': ROOT / 'data' / 'results' / 'evaluation_pe.csv',
}

for run_name, baseline_path in mapping.items():
    new_path = OUT_ROOT / f'postprocess_{run_name}' / 'evaluation.csv'
    if not baseline_path.exists() or not new_path.exists():
        continue

    b = pd.read_csv(baseline_path)
    n = pd.read_csv(new_path)

    b_ev = b[b['n_metrics'] > 0].copy()
    n_ev = n[n['n_metrics'] > 0].copy()

    b_pass = int(b_ev['passed'].fillna(False).sum())
    n_pass = int(n_ev['passed'].fillna(False).sum())
    b_avg = float(b_ev['score'].mean()) if len(b_ev) else 0.0
    n_avg = float(n_ev['score'].mean()) if len(n_ev) else 0.0

    delta_rows.append({
        'run': run_name,
        'baseline_passed': b_pass,
        'new_passed': n_pass,
        'delta_passed': n_pass - b_pass,
        'baseline_avg_score': b_avg,
        'new_avg_score': n_avg,
        'delta_avg_score': n_avg - b_avg,
    })

delta_df = pd.DataFrame(delta_rows).sort_values('delta_avg_score', ascending=False)
display(Markdown('## Delta vs baseline'))
display(delta_df)

## Delta vs baseline

,run,baseline_passed,new_passed,delta_passed,baseline_avg_score,new_avg_score,delta_avg_score
0,pertable_refcats,6,6,0,0.303107,0.303107,0.0
1,mst_refcats,6,6,0,0.328326,0.328326,0.0
2,pe_refcats,2,2,0,0.149866,0.149866,0.0
